# Vektorisierung der Beschreibung

In [1]:
import os
from src.dataPipeline import DataPipeline
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

os.chdir('..')


columns_to_drop_all = [
'Unnamed: 0.1', 'Unnamed: 0','Municipality','Living space',
'Plot area','Floor space','location',
'description',
#'detailed_description',
'url','table','detail_responsive#municipality',
'detail_responsive#surface_living','detail_responsive#floor','detail_responsive#available_from',
'Gemeinde','Wohnfläche','Stockwerk','Nutzfläche','Verfügbarkeit','Grundstücksfläche',
'detail_responsive#surface_property','Commune','Surface habitable','Surface du terrain',
'Surface utile','Disponibilité','Étage','Comune','Superficie abitabile','Disponibilità',
'Piano','Superficie del terreno','Superficie utile','Municipality_merged',
'Floor_merged','Living_space_merged','Plot_area_merged','Availability_merged','location_parsed',
'title',
'price','address','link','details_structured','index',
'Locality','Zip','rooms','Floor_unified','Living_area_unified','space',
'price_s','address_s','Surface living:','Land area:',
'description_detailed','Floor space:','Volume:','plz','Number of toilets:','Gross yield:',
'Minimum floor space:','space_cleaned'

# Temporary columns
, 'Gross return'
, 'details'
, 'Room height:'
, 'features'
, 'type'
, 'provider' ,
'type_unified',
'Availability',
'Floor',
'detail_responsive#surface_usable',
'Floor_space_merged',
'lat',
'lon',
'ForestDensityL',
'ForestDensityM',
'ForestDensityS',
'Latitude',
'Longitude',
'NoisePollutionRailwayL',
'NoisePollutionRailwayM',
'NoisePollutionRailwayS',
'NoisePollutionRoadL',
'NoisePollutionRoadM',
'NoisePollutionRoadS',
'PopulationDensityL',
'PopulationDensityM',
'PopulationDensityS',
'RiversAndLakesL',
'RiversAndLakesM',
'RiversAndLakesS',
'WorkplaceDensityL',
'WorkplaceDensityM',
'WorkplaceDensityS',
'distanceToTrainStation',
'gde_area_agriculture_percentage',
'gde_area_forest_percentage',
'gde_area_nonproductive_percentage',
'gde_area_settlement_percentage',
'gde_average_house_hold',
'gde_empty_apartments',
'gde_foreigners_percentage',
'gde_new_homes_per_1000',
'gde_politics_bdp',
'gde_politics_cvp',
'gde_politics_evp',
'gde_politics_fdp',
'gde_politics_glp',
'gde_politics_gps',
'gde_politics_pda',
'gde_politics_rights',
'gde_politics_sp',
'gde_politics_svp',
'gde_pop_per_km2',
'gde_population',
'gde_private_apartments',
'gde_social_help_quota',
'gde_tax',
'gde_workers_sector1',
'gde_workers_sector2',
'gde_workers_sector3',
'gde_workers_total',
'plz_parsed',
'No. of rooms:',
'Number of apartments:',
'Last refurbishment:',
'Number of floors:',
'Year built:',
'Space extracted',
'Plot_area_unified',
]

dp = DataPipeline()
dp.readCsv("data/immo_data_202208_v2.csv")
dp.dropColumns(columns_to_drop_all)

df = dp.getData()
df

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:45: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)


,detailed_description,price_cleaned
0,DescriptionLuxuriöse Attika-Wohnung direkt an ...,1150000.0
1,DescriptionStilvolle Liegenschaft an ruhiger L...,1420000.0
2,"detail_responsive#description_title2,5 Zimmerw...",720000.0
3,DescriptionDieses äusserst grosszügige Minergi...,1430000.0
4,DescriptionAus ehemals zwei Wohnungen wurde ei...,995000.0
...,...,...
22476,"Description\n""Hausteil mit verschieden Nutzung...",475000.0
22477,"Description\n""J'ADORE - Exklusives Wohnen in W...",1490000.0
22478,"Description\n""Einmalige Gelegenheit an sehr gu...",1450000.0
22479,"Description\n""LA VIE - Exklusives Wohnen in Ma...",1290000.0


In [2]:
# set the random seed to get comparable results
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

np.random.seed(0)

features = df['detailed_description']
# Fill NaN values with empty string
features = features.fillna('')
target = df['price_cleaned']

# Drop all columns with NaN values in price_cleaned
features = features[target.notnull()]
target = target[target.notnull()]

In [3]:

# prepare the data
comments_train, comments_test, y_train, y_test = train_test_split(features, target, test_size = 0.3) 

# we use a count vectorizer to break the individual comments into tokens, one token per word
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# we create the training a matrix 
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# train the classifier
#classifier = LogisticRegression(max_iter=2000)
#classifier.fit(X_train, y_train)
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., 'œnothèque', 'œuvre', 'œuvres'],
      dtype=object)

In [4]:
from sklearn.ensemble import RandomForestRegressor

classifier = RandomForestRegressor()
classifier.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
#test the data
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

#test the classifier with mape
from sklearn.metrics import mean_absolute_percentage_error
y_pred = classifier.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print("MAPE:", mape)

In [ ]:
#plot the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test, classifier.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Embedding for Text with BERT

In [ ]:
from transformers import BertModel, BertTokenizer
import torch.nn as nn

class BertForRegression(nn.Module):
    def __init__(self, pretrained_model_name):
        super(BertForRegression, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 10)
        self.fc2 = nn.Linear(10, 1)
        for param in self.bert.parameters():
            param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output  # `[CLS]` token output
        x = self.fc1(cls_output)
        x = nn.ReLU()(x)
        return self.fc2(x)
    

In [ ]:
from torch.utils.data import Dataset

class RealEstateDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        description = self.dataframe.iloc[index]['detailed_description']
        price = self.dataframe.iloc[index]['price_cleaned']

        # Tokenisierung
        encoding = self.tokenizer(
            description,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Entferne Batch-Dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'target': torch.tensor(price, dtype=torch.float)
        }

In [ ]:
from torch.utils.data import DataLoader
from src import torchModelRun
import torch

import pandas as pd
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

max_length = 128
train_dataset = RealEstateDataset(train_df, tokenizer, max_length)
test_dataset = RealEstateDataset(test_df, tokenizer, max_length)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)




#import sgd
from torch.optim import SGD
import torch.nn as nn

# Modell initialisieren
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForRegression('bert-base-uncased').to(device)

# Optimizer und Verlustfunktion
optimizer = SGD(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

# Training
epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target = batch['target'].view(-1, 1).to(device)
        
        optimizer.zero_grad()
        output = model(input_ids, attention_mask)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.4f}")




# Versuch mit Huggingface Library 2

In [2]:
import numpy as np
import pandas as pd
from datasets import Dataset

df = df[df['price_cleaned'].notnull()]  # Entferne Zeilen mit fehlendem Preis
df = df[df['detailed_description'].notnull()]  # Entferne Zeilen mit fehlender Beschreibung
df['log_price'] = np.log(df['price_cleaned'])  # Logarithmiere den Preis

# Erstelle HuggingFace Dataset
data = Dataset.from_pandas(df[['detailed_description', 'log_price']].rename(columns={
    'detailed_description': 'text', 'log_price': 'label'
}))

In [3]:
from transformers import BertTokenizer


model_name = "bert-base-german-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Preprocessing-Funktion
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=64)

# Tokenisierung anwenden
dataset = data.map(preprocess_function, batched=True)

# Datenset splitten
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

In [4]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch
import torch.nn as nn

# Modell laden
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)  # num_labels=1 für Regression

# BERT einfrieren
for param in model.bert.parameters():
    param.requires_grad = False
    
model.classifier = nn.Linear(model.bert.config.hidden_size, 1)

# Metriken für Regression
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    mse = ((predictions - labels) ** 2).mean()
    rmse = np.sqrt(mse)
    mape = (np.abs(predictions - labels) / labels).mean() * 100
    return {"mse": mse, "rmse": rmse, "mape": mape}

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=1,
)

# Trainer initialisieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Modell trainieren
trainer.train()

# Modell speichern
trainer.save_model("./regression_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\denis\AppData\Local\Temp\ipykernel_42284\1745905395.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wand

Epoch,Training Loss,Validation Loss


In [29]:
# load model
from transformers import BertForSequenceClassification, BertTokenizer

model_name = "bert-base-german-cased"
model = BertForSequenceClassification.from_pretrained("./regression_model")
tokenizer = BertTokenizer.from_pretrained(model_name)

# Test the model with new data
new_texts = ["Alte Scheune in ländlicher Umgebung zu verkaufen. Ist sanierungsbedürftig. Kiesweg führt zum Haus.adfasdfasdfasdf viel text"]
tokenized = tokenizer(new_texts, truncation=True, padding="max_length", max_length=64, return_tensors="pt")
model.eval()
with torch.no_grad():
    predictions = model(**tokenized).logits
    print("Predictions:", np.exp(predictions.squeeze().numpy()))

Predictions: 1274757.8
